<a href="https://colab.research.google.com/github/saiku122/AIJobcolle/blob/master/MachineLearning/python/exercise/02_work/classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/saiku122/AIJobcolle.git

Cloning into 'AIJobcolle'...
remote: Enumerating objects: 272, done.
remote: Counting objects: 100% (272/272), done.
remote: Compressing objects: 100% (181/181), done.
remote: Total 272 (delta 103), reused 155 (delta 46), pack-reused 0
Receiving objects: 100% (272/272), 11.70 MiB | 16.66 MiB/s, done.
Resolving deltas: 100% (103/103), done.


In [5]:
cd /content/AIJobcolle/MachineLearning/python

/content/AIJobcolle/MachineLearning/python


In [6]:
cd ./exercise

/content/AIJobcolle/MachineLearning/python/exercise


In [7]:
cd ./02_work/

/content/AIJobcolle/MachineLearning/python/exercise/02_work


In [16]:
pwd

'/content/AIJobcolle/MachineLearning/python/exercise/02_work'

In [13]:
#ライブラリインポート
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import RFECV
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
import joblib

**①データの前処理**

In [14]:
 # SET PARAMETERS
file_model = 'dm_for_model'
file_score = 'dm_for_fwd'
ohe_cols = ['mode_category']

# CLASSIFIER
model_name = 'GBC_001'
clf = Pipeline([('scl',StandardScaler()), ('est',GradientBoostingClassifier(random_state=1))])

In [23]:
#データ取り込み
df = pd.read_csv('./data/'+ file_model + '.csv', header=0)
ID = df.iloc[:,0] 
y = df.iloc[:,-1]
X = df.iloc[:,1:-1]

In [33]:
#  preprocessing-1:one-hot encoding処理
X_ohe = pd.get_dummies(X, dummy_na=True, columns=ohe_cols) # mode_category列をダミー変数化（NAも列追加）
X_ohe = X_ohe.dropna(axis=1, how='all')#すべての値が欠損値である行・列を削除
X_ohe_columns = X_ohe.columns.values

In [ ]:
###@@@変数チェック(処理関係なし）@@@
X_ohe.columns.values

In [45]:
# preprocessing-2: 欠損値の置き換え ※一部naの列はのこっているため必要
imp = SimpleImputer() #平均値で欠損地(Nan)を補完
imp.fit(X_ohe)        #ワンホットエンコーディングの結果に対して欠損地置き換えした統計値を保存
X_ohe = pd.DataFrame(imp.transform(X_ohe), columns=X_ohe_columns) #X_ohe_columに対して変換処実行

In [49]:
# preprocessing-3: 特徴量を抽出
selector = RFECV(estimator=RandomForestClassifier(n_estimators=100,random_state=0), step=0.05) #特徴量をRFE(主成分分析)を用いて次元削減する。(モデルはランダムフォレスト)
selector.fit(X_ohe, y) #RFEした結果を保存
X_ohe_selected = selector.transform(X_ohe) #X_oheに反映
X_ohe_selected = pd.DataFrame(X_ohe_selected, columns=X_ohe_columns[selector.support_]) #削減実行後のデータを再構成

(717, 318)

In [50]:
###@@@特徴量抽出前後のデータ量確認（処理には関係なし）@@@
display(X_ohe.shape)
display(X_ohe_selected.shape)

(717, 334)

(717, 318)

以降、テストデータは「X_ohe_selected」を用いる

In [63]:
# preprocessing-4: スコア評価用のデータをロード　※基本的には前段の焼き増し
dfs = pd.read_csv('./data/'+ file_score + '.csv', header=0)
IDs = dfs.iloc[:,[0]] 
Xs = dfs.iloc[:,1:-1]
Xs_ohe = pd.get_dummies(Xs, dummy_na=True, columns=ohe_cols)
cols_m = pd.DataFrame(None, columns=X_ohe_columns, dtype=float) #X_ohe_columnsで抽出された特徴量を抽出

In [64]:
#テストデータと正解データのラベルを合わせる必要があるため本処理が存在。
Xs_exp = pd.concat([cols_m, Xs_ohe])
Xs_exp.loc[:,list(set(X_ohe_columns)-set(Xs_ohe.columns.values))] = \
    Xs_exp.loc[:,list(set(X_ohe_columns)-set(Xs_ohe.columns.values))].fillna(0, axis=1)
Xs_exp = Xs_exp.drop(list(set(Xs_ohe.columns.values)-set(X_ohe_columns)), axis=1)

In [61]:
Xs_exp = Xs_exp.reindex(X_ohe_columns, axis=1)
Xs_exp = pd.DataFrame(imp.transform(Xs_exp), columns=X_ohe_columns)
Xs_exp_selected = Xs_exp.loc[:, X_ohe_columns[selector.support_]] #再度RFEをやるのではなくテストデータの結果にあわせる

In [62]:
cols_m

,trips,amount_of_yen,quantity,purchase_period,purchase_recency,purchase_amount_per_trips,regularity,trips_cancellation,trips_cat85099,trips_cat85123,trips_cat22423,trips_cat47566,trips_cat84879,trips_cat20725,trips_cat22720,trips_catpost,trips_cat23203,trips_cat22383,mode_category_15044,mode_category_15056,mode_category_20718,mode_category_20725,mode_category_20727,mode_category_20728,mode_category_20971,mode_category_21034,mode_category_21084,mode_category_21116,mode_category_21136,mode_category_21137,mode_category_21181,mode_category_21216,mode_category_21313,mode_category_21314,mode_category_21326,mode_category_21340,mode_category_21417,mode_category_21466,mode_category_21495,mode_category_21519,...,mode_category_85036,mode_category_85039,mode_category_85040,mode_category_85049,mode_category_85059,mode_category_85066,mode_category_85071,mode_category_85099,mode_category_85114,mode_category_85116,mode_category_85118,mode_category_85123,mode_category_85131,mode_category_85132,mode_category_85141,mode_category_85152,mode_category_85159,mode_category_85169,mode_category_85170,mode_category_85175,mode_category_85178,mode_category_85180,mode_category_85183,mode_category_85186,mode_category_85194,mode_category_85202,mode_category_85206,mode_category_85212,mode_category_85230,mode_category_85231,mode_category_85232,mode_category_90125,mode_category_90200,mode_category_90201,mode_category_90214,mode_category_C2,mode_category_D,mode_category_M,mode_category_POST,mode_category_nan


**②モデリング**  
※最初にGBCを定義済み

In [73]:
clf.fit(X_ohe_selected, y.values.ravel())#Xを標準化、勾配ブースティングを適用)
#joblib.dump(clf, './03_output/'+ model_name + '.pkl')
results = cross_val_score(clf, X_ohe_selected, y, scoring='roc_auc', cv=5)
print('cv score:', np.average(results), '+-', np.std(results))

cv score: 0.7198053221288514 +- 0.02126526880177877


**score評価**

In [74]:
score = pd.DataFrame(clf.predict_proba(Xs_exp_selected)[:,1], columns=['pred_score'])
IDs.join(score).to_csv('./data/'+  model_name + '_' + file_score + '_with_pred.csv', index=False)

In [75]:
imp = pd.DataFrame([clf.named_steps['est'].feature_importances_], columns=X_ohe_columns[selector.support_])
imp.T.to_csv('./data/'+  model_name + '_feature_importances.csv', index=True)